In [ ]:
using Plots
using LinearAlgebra
using Printf
using LaTeXStrings
using DifferentialEquations

In [ ]:
default(lw=2,markersize = 6,
    xtickfont=font(12), ytickfont=font(12), 
    guidefont=font(14), legendfont=font(12),titlefont=font(12))

# Advection Equation
Solve
$$
u_t + c u_x = 0,
$$
on the periodic domain $[0,L)$ and $c$ is a constant.

In [ ]:
L = 10;
c = 1.;

N = 50;
x = LinRange(0, L,N+1)[1:end-1];
@show Δx = x[2] - x[1];
Δt = 1.999*Δx/(2*c);
n_steps = 100;

u0 = @. exp( -(x- L/2)^2);
plot(x, u0,label="")
xlabel!(L"$x$")
ylabel!(L"$u_0$")

In [ ]:
function integrate_advection_equation1(u0, c, Δx, Δt, n_steps)
    u = deepcopy(u0);
    t = 0.
    
    u_traj = [deepcopy(u)];
    t_traj = Float64[t];
    
    u_new = similar(u);
    
    N = length(u);
    
    for n in 1:n_steps
        # update u
        for j in 1:N-1
            u_new[j] = u[j] - c * Δt/Δx * (u[j+1]-u[j]);
        end
        u_new[end] = u[end] - c * Δt/Δx * (u[1]-u[end]); # for periodic BCs
        # copy over
        @. u = u_new;
        
        t  += Δt;
        
        push!(u_traj, deepcopy(u));
        push!(t_traj, t);
    end

    return t_traj, u_traj

end

In [ ]:
t_traj, u_traj = integrate_advection_equation1(u0, c, Δx, Δt, n_steps);

In [ ]:
anim = @animate for (n,t) in enumerate(t_traj)
    plot(x, u_traj[n], label="");
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$");
end 

In [ ]:
gif(anim, fps = 15)


In [ ]:
function integrate_advection_equation2(u0, c, Δx, Δt, n_steps)
    u = deepcopy(u0);
    t = 0.
    
    u_traj = [deepcopy(u)];
    t_traj = Float64[t];
    
    u_new = similar(u);
    
    N = length(u);
    
    for n in 1:n_steps
        # update u
        u_new[1] = u[1] - c * Δt/Δx * (u[1]-u[end-1]); # for periodic boundary conditions
        for j in 2:N
            u_new[j] = u[j] - c * Δt/Δx * (u[j]-u[j-1]);
        end
        # copy over
        @. u = u_new;
        
        t  += Δt;
        
        push!(u_traj, deepcopy(u));
        push!(t_traj, t);
    end

    return t_traj, u_traj

end

In [ ]:
# Δt = 0.01;
# n_steps = 1000;

t_traj, u_traj = integrate_advection_equation2(u0, c, Δx, Δt, n_steps);

In [ ]:
anim = @animate for (n,t) in enumerate(t_traj)
    plot(x, u_traj[n], label="");
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$");
    # ylims!(0,1);
end 

In [ ]:
gif(anim, fps = 60)


In [ ]:
function fadvection!(du, u, p, t)
    c = p[1];
    Δx = p[2];
    du[1] = - c/Δx * (u[1]-u[end-1]);
    N = length(u);
    for j in 2:N
        du[j] = - c/Δx * (u[j]-u[j-1]);
    end
    du
end

In [ ]:
p = [c, Δx]
tspan = (0, 20);
prob = ODEProblem(fadvection!, u0, tspan, p);


In [ ]:
t_vals= 0:1:20;

sol = solve(prob, saveat=t_vals);

In [ ]:
anim = @animate for (t, u) in zip(sol.t, sol.u)
    plot(x, u, label="")
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$")
    ylims!(0,1)
end

In [ ]:
gif(anim, fps = 15)
